In [1]:
%pip install sqlalchemy
%pip install sympy


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from sqlalchemy.dialects.mysql.mariadb import loader
from sympy.codegen.fnodes import dimension
%pip install langchain_community



Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\VsCodeProjects\Quran-Hadith-Bot\backend


In [4]:
# from langchain_community.document_loaders.csv_loader import CSVLoader
# loader=CSVLoader(file_path="/content/all_hadiths_clean.csv",encoding="utf-8")
# docs=loader.load()
# print(len(docs))


In [5]:
# print(type(docs))
# print(len(docs[444].page_content.split()))
# print(docs[444])

In [6]:
# def countTokens(text):
#     return len(text.split())
# maxTokens = 0
# maxIndex = -1
# for i, doc in enumerate(docs):
#     token_count = countTokens(doc.page_content)
#     if token_count > maxTokens:
#         maxTokens = token_count
#         maxIndex = i

# print(maxTokens)


In [7]:
# print(token_count)

In [8]:
%pip install --upgrade --quiet  langchain langchain-huggingface sentence_transformers


Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install tiktoken

In [10]:
!pip install pandas

QURAN DATA processing
Loading
creating document a=object manually {adding metadata and content}
chunks creation for bigger ayahs that might gets truncated if not within token limit

In [11]:
import pandas as pd
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
import math

def clean_value(value):
    if value is None:
        return 0
    if isinstance(value, float) and pd.isna(value):  # catches NaN
        return 0
    if str(value).lower() in ["nan", "null", "none"]:
        return 0
    return value

# 1. Load the CSV file into a Pandas DataFrame
df = pd.read_csv('The Quran Dataset.csv')

# 2. Create a list of LangChain Document objects manually
docs = []
for index, row in df.iterrows():

    # Construct the metadata dictionary
    metadata = {
    'surah_no': clean_value(row.get('surah_no')),
    'surah_name_en': clean_value(row.get('surah_name_en')),
    'surah_name_ar': clean_value(row.get('surah_name_ar')),
    'surah_name_roman': clean_value(row.get('surah_name_roman')),
    'ayah_no_surah': clean_value(row.get('ayah_no_surah')),
    'ayah_no_quran': clean_value(row.get('ayah_no_quran')),
    'ayah_ar': clean_value(row.get('ayah_ar')),
    'ayah_en': clean_value(row.get('ayah_en')),
    'ruko_no': clean_value(row.get('ruko_no')),
    'juz_no': clean_value(row.get('juz_no')),
    'manzil_no': clean_value(row.get('manzil_no')),
    'hizb_quarter': clean_value(row.get('hizb_quarter')),
    'total_ayah_surah': clean_value(row.get('total_ayah_surah')),
    'total_ayah_quran': clean_value(row.get('total_ayah_quran')),
    'place_of_revelation': clean_value(row.get('place_of_revelation')),
    'sajdah_ayah': clean_value(row.get('sajah_ayah')),
    'sajdah_no': clean_value(row.get('sajdah_no')),
    'no_of_word_ayah': clean_value(row.get('no_of_word_ayah')),
    'list_of_words': clean_value(row.get('list_of_words'))
}

     # Construct the page_content
    # This will be the text we want the embeddings to represent.
    page_content = f"AyahArabic: {row['ayah_ar']}\nAyahEnglish: {row['ayah_en']}"

    # Create the Document object and add it to our list
    docs.append(Document(page_content=page_content, metadata=metadata))

# 3. Now, pass this correctly structured list of documents to the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# This will split your documents correctly and preserve the metadata
all_Quran_chunks = text_splitter.split_documents(docs)

# You can now check the size and content of your new chunks
print(f"Number of chunks: {len(all_Quran_chunks)}")
print("Example chunk:",all_Quran_chunks[2])


Number of chunks: 6919
Example chunk: page_content='AyahArabic: ٱلرَّحْمَٰنِ ٱلرَّحِيمِ
AyahEnglish: the Most Compassionate, Most Merciful,' metadata={'surah_no': 1, 'surah_name_en': 'The Opener', 'surah_name_ar': 'الفاتحة', 'surah_name_roman': 'Al-Fatihah', 'ayah_no_surah': 3, 'ayah_no_quran': 3, 'ayah_ar': 'ٱلرَّحْمَٰنِ ٱلرَّحِيمِ', 'ayah_en': 'the Most Compassionate, Most Merciful,', 'ruko_no': 1, 'juz_no': 1, 'manzil_no': 1, 'hizb_quarter': 1, 'total_ayah_surah': 7, 'total_ayah_quran': 6236, 'place_of_revelation': 'Meccan', 'sajdah_ayah': False, 'sajdah_no': 0, 'no_of_word_ayah': 2, 'list_of_words': '[ٱلرَّحْمَٰنِ,ٱلرَّحِيمِ]'}


Hadith data processing

In [12]:
import pandas as pd
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 1. Load the CSV file into a Pandas DataFrame
df = pd.read_csv('all_hadiths_clean.csv')

# 2. Create a list of LangChain Document objects manually
docs = []
for index, row in df.iterrows():
    # Construct the metadata dictionary
    metadata = {
        'id': row['id'],
        'hadith_id': row['hadith_id'],
        'source': row['source'],
        'chapter_no': row['chapter_no'],
        'hadith_no': row['hadith_no'],
        'chapter': row['chapter'],
    }

    # Construct the page_content
    # This will be the text we want the embeddings to represent.
    page_content = f"Arabic: {row['text_ar']}\nEnglish: {row['text_en']}"

    # Create the Document object and add it to our list
    docs.append(Document(page_content=page_content, metadata=metadata))

# 3. Now, pass this correctly structured list of documents to the splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

# This will split your documents correctly and preserve the metadata
all_hadith_chunks = text_splitter.split_documents(docs)

# You can now check the size and content of your new chunks
print(f"Number of chunks: {len(all_hadith_chunks)}")
print("Example chunk:")


Number of chunks: 76880
Example chunk:


In [13]:
!pip install -qU langchain langchain-pinecone langchain-openai


In [14]:
!pip install dotenv

In [15]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
load_dotenv()
# This line checks if the Pinecone API key exists as a Colab Secret.
# If you don't have a secret named PINECONE_API_KEY, it will raise an error.
try:
    pinecone_api_key = os.getenv("PINECONE_API_KEY")
except Exception as e:
    raise ValueError("'PINECONE_API_KEY' not found. Please add your key to the Secrets panel.") from e

pc = Pinecone(api_key=pinecone_api_key)

In [16]:
!pip install huggingface_hub
%pip install --upgrade --quiet  langchain langchain-huggingface sentence_transformers

Note: you may need to restart the kernel to use updated packages.


Hadith index initialization in pinecone

In [17]:

from pinecone import  ServerlessSpec

# 1. Initialize the Pinecone client
# This uses your API key and environment to connect.


# 2. Define your index parameters
index_name = "hadith-rag-index" # Choose a unique, descriptive name
dimension = 1024 # Match the output dimension of your embedding model
metric = "cosine" # Cosine similarity is a good default for semantic search

# 3. Check if the index already exists
if index_name not in pc.list_indexes().names():
    print(f"Creating a new Pinecone index: {index_name}...")
    pc.create_index(
        name=index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("Index created successfully!")
else:
    print(f"Index '{index_name}' already exists.")

# 4. Now you have an active index that you can connect to.
pinecone_index = pc.Index(index_name)

Index 'hadith-rag-index' already exists.


Hadith embeddings->pinecone index
Adding embeddings of hadith chunks to the pinecone index (hadith rag index)

In [18]:
###CODE TO create and store embeddings in the vector store
###commennted so that i dont accidentally recreate embeddings
#=========######################################=============
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_pinecone import PineconeVectorStore
# embeddingModel= HuggingFaceEmbeddings(model_name="omarelshehy/arabic-english-sts-matryoshka-v2.0")

# vectorStore=PineconeVectorStore(index=pinecone_index,embedding=embeddingModel)
# vectorStore.add_documents(documents=all_hadith_chunks)

In [19]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()
# Get API key from Colab secrets
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = "hadith-rag-index" # You should set your index name here

# Set the Pinecone API key as an environment variable
os.environ["PINECONE_API_KEY"] = pinecone_api_key

# 1. Initialize your embedding model
embeddingModel = HuggingFaceEmbeddings(model_name="omarelshehy/arabic-english-sts-matryoshka-v2.0")

# 2. Re-initialize the vector store by connecting to the existing index
# Note: The `from_existing_index` method does NOT take an API key as a parameter,
# but relies on the PINECONE_API_KEY environment variable.
vectorStore = PineconeVectorStore.from_existing_index(

    index_name=pinecone_index_name,
    embedding=embeddingModel
)

# # You can now perform a similarity search with the vectorStore object
# query = "how to pray namaz according to hadith"
# results = vectorStore.similarity_search(query, k=5)

# print(results)

Quran index initialization in pinecone

In [20]:

from pinecone import  ServerlessSpec

# 1. Initialize the Pinecone client
# This uses your API key and environment to connect.


# 2. Define your index parameters
Quran_index_name = "quran-rag-index" # Choose a unique, descriptive name
dimension = 1024 # Match the output dimension of your embedding model
metric = "cosine" # Cosine similarity is a good default for semantic search

# 3. Check if the index already exists
if Quran_index_name not in pc.list_indexes().names():
    print(f"Creating a new Pinecone index: {Quran_index_name}...")
    pc.create_index(
        name=Quran_index_name,
        dimension=dimension,
        metric=metric,
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("Index created successfully!")
else:
    print(f"Index '{Quran_index_name}' already exists.")

# 4. Now you have an active index that you can connect to.
pinecone_Quran_index = pc.Index(Quran_index_name)

Index 'quran-rag-index' already exists.


below code is run only once in his lifetime , as it will simply add embeddings to the index 
running it every time will keep adding indexes over indexes i.e repeating quran or hadith embeddings

In [21]:
# ###CODE TO create and store embeddings in the vector store
# ###commented so that i dont accidentally recreate embeddings
# #=========######################################=============
# from langchain_huggingface import HuggingFaceEmbeddings
# from langchain_pinecone import PineconeVectorStore
# embeddingModel= HuggingFaceEmbeddings(model_name="omarelshehy/arabic-english-sts-matryoshka-v2.0")

# Quran_vectorStore=PineconeVectorStore(index=pinecone_Quran_index,embedding=embeddingModel)
# Quran_vectorStore.add_documents(documents=all_Quran_chunks)

below code is to initialize index after that we have created the index so that we dont have to run the above code everytime

In [22]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv()
# Get API key from Colab secrets
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pinecone_index_name = "quran-rag-index" # You should set your index name here

# Set the Pinecone API key as an environment variable
os.environ["PINECONE_API_KEY"] = pinecone_api_key

# 1. Initialize your embedding model
embeddingModel = HuggingFaceEmbeddings(model_name="omarelshehy/arabic-english-sts-matryoshka-v2.0")

# 2. Re-initialize the vector store by connecting to the existing index
# Note: The `from_existing_index` method does NOT take an API key as a parameter,
# but relies on the PINECONE_API_KEY environment variable.
Quran_vectorStore = PineconeVectorStore.from_existing_index(

    index_name=pinecone_index_name,
    embedding=embeddingModel
)

# # You can now perform a similarity search with the vectorStore object
# query = "how to pray namaz according to hadith"
# results = vectorStore.similarity_search(query, k=5)

# print(results)

In [23]:
Quran_vectorStore.similarity_search("morality")

[Document(id='6feec282-09db-4615-83c9-dd55427a3ddd', metadata={'ayah_ar': '۞ إِنَّ ٱللَّهَ يَأْمُرُ بِٱلْعَدْلِ وَٱلْإِحْسَٰنِ وَإِيتَآئِ ذِى ٱلْقُرْبَىٰ وَيَنْهَىٰ عَنِ ٱلْفَحْشَآءِ وَٱلْمُنكَرِ وَٱلْبَغْىِ ۚ يَعِظُكُمْ لَعَلَّكُمْ تَذَكَّرُونَ', 'ayah_en': 'Indeed, Allah commands justice, grace, as well as courtesy to close relatives. He forbids indecency, wickedness, and aggression. He instructs you so perhaps you will be mindful.', 'ayah_no_quran': 1991.0, 'ayah_no_surah': 90.0, 'hizb_quarter': 111.0, 'juz_no': 14.0, 'list_of_words': '[۞,إِنَّ,ٱللَّهَ,يَأْمُرُ,بِٱلْعَدْلِ,وَٱلْإِحْسَٰنِ,وَإِيتَآئِ,ذِى,ٱلْقُرْبَىٰ,وَيَنْهَىٰ,عَنِ,ٱلْفَحْشَآءِ,وَٱلْمُنكَرِ,وَٱلْبَغْىِ,ۚ,يَعِظُكُمْ,لَعَلَّكُمْ,تَذَكَّرُونَ]', 'manzil_no': 3.0, 'no_of_word_ayah': 18.0, 'place_of_revelation': 'Meccan', 'ruko_no': 236.0, 'sajdah_ayah': 0.0, 'sajdah_no': 0.0, 'surah_name_ar': 'النحل', 'surah_name_en': 'The Bee', 'surah_name_roman': 'An-Nahl', 'surah_no': 16.0, 'total_ayah_quran': 6236.0, 'total_ayah_s

In [24]:
%pip install -U langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [25]:

# import os
# from google.colab import userdata
# from langchain_google_genai import ChatGoogleGenerativeAI

# # Retrieve your API key from Colab secrets
# os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

# # Initialize the Gemini model
# llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)

# # Now you can use the 'llm' object to interact with the model
# response = llm.invoke("What is a Large Language Model?")
# print(response.content)

In [26]:
# from langchain.chains import RetrievalQA

# # Assuming you have successfully run the previous cells to initialize these
# # llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)
# # vectorStore = PineconeVectorStore.from_existing_index(...)

# # 1. Create a retriever from your vector store
# retriever = vectorStore.as_retriever()

# # 2. Create the RetrievalQA chain
# qa_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff", # This packs all the retrieved documents into one prompt
#     retriever=retriever
# )

# # 3. Use the chain to get a final, synthesized answer
# query = "What is the hadith about the best charity?"
# result = qa_chain.invoke(query)

# # The result is a dictionary; the answer is in the 'result' key
# print(result['result'])

In [27]:
# from ast import parse
# from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
# from langchain.chains import RetrievalQA

# parser=StrOutputParser()
# question="What is the hadith about the best charity?"
# contextRetriever=vectorStore.similarity_search(question,k=5)
# prompt=ChatPromptTemplate(
#     messages=[
#         SystemMessagePromptTemplate.from_template("You are a helpful assistant who answers questions about the hadith from the given context{context} ;try to keep answer brief unless asked by the user , provide complete hadith if user specifically ask about haditha nd later exapplain that hadith as per users question.Also whenever you quote a hadith show hadith number source and chapter from the metadata,and as per user request do show user arabic text of the hadith"),

#         HumanMessagePromptTemplate.from_template("{question}")
#     ]
# )
# chain=prompt|llm|parser
# chain.invoke({"context":context,"question":question})

In [28]:
!pip install langchain-core langgraph>0.2.27

!pip install langgraph-checkpoint-sqlite

In [29]:
from re import S
import os
from langchain_core.output_parsers import StrOutputParser
import sqlite3
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END, START
from langgraph.checkpoint.sqlite import SqliteSaver
from typing import TypedDict, Annotated, Sequence
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from typing import Optional
from langchain.memory import ConversationBufferWindowMemory

load_dotenv()

# # Retrieve your API key from Colab secrets
# os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

# Initialize the Gemini model
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)
parser=StrOutputParser()
prompt=ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("""""You are **ISLAM AI ASSISTANT**.  
Your rules:  
- Only answer from the provided **context: {context}**.  
- Apply the **filter: {Filter}** → only use sources allowed by the filter.  
- If no answer exists in the context → respond with an apology.  
- Provide metadata:  
  - If **Quran** → show Arabic + English + surah/ayah reference.  
  - If **Hadith** → show English text + arabic + book + hadith number + grade.  
- Answers must be **brief, direct, and to the question**.  
- Never use your own knowledge or external data.  
- Use past **message history** for consistency.  
- Never provide Quran or Hadith outside the context.
- In case where answer in hadith or quran or both is not found , mark all the field of it as none and in summary apologize i couldnt find the answer                                                   
- summary should be a direct answer to the question based on the ayah and hadith in your response , always give **verified scholarly analysis** .  
                                                   

Always  strictly format output in the following JSON format:  


{{
  "summary": "direct answering the user's question",
  "references": [
    {{
      "type": "Quran",
      "surah": "Al-Ikhlas",
      "ayahNumber": "1-4",
      "ayahArabic": "قُلْ هُوَ ٱللَّهُ أَحَدٌ",
      "ayahEnglish": "Say: He is Allah, One.",
      "reference": "Quran 112:1-4"
    }},
    {{
      "type": "Hadith",
      "book": "Sahih al-Bukhari",
      "hadithNumber": "13",
      "grade": "Sahih",
      "textArabic": "إِنَّمَا الأَعْمَالُ بِالنِّيَّاتِ",
      "textEnglish": "Actions are judged by intentions."
    }}
  ]
}}
"""""),

        MessagesPlaceholder(variable_name="messages")
    ]
)

class RAGState(TypedDict):#Typeddictionary is specified dictionary where we specify attributes and their type hints(value types)
    # This is our shared state schema
    messages: Annotated[list, lambda x, y: x + y]#here we specify messages to be the list and it would keep appending user messages
    documents: Sequence[Document]
    filters: Annotated[Optional[str], lambda x, y: y or x]




def retrieveDocuments(state: RAGState):
    # This node reads the latest message from the state
    question = state["messages"][-1].content
    # ... then performs retrieval ...

    if state["filters"] == "Quran":
        retrieved_docs=Quran_vectorStore.similarity_search(question,k=5)
    elif state["filters"] =="Ahadith":
        retrieved_docs=vectorStore.similarity_search(question,k=5)
    else:
        
        AhadithRetrieved=vectorStore.similarity_search(question,k=5)
        QuranRetrieved=Quran_vectorStore.similarity_search(question,k=5)
        retrieved_docs = AhadithRetrieved + QuranRetrieved



    # ... and returns the updates to the state
    return {"documents": retrieved_docs}



def generate_answer(state: RAGState):
    # This node reads both the messages and the documents from the state
    messages = state["messages"]
    documents = state["documents"]
    ffilter=state["filters"]
     
    if len(messages) > 10:
        messages = messages[-10:]  
        state["messages"] = messages

     #  Limit documents to last 5
    if len(documents) > 5:
        documents = documents[-5:]
        state["documents"] = documents


    # ... then performs LLM generation with a prompt template ...

    full_prompt = prompt.partial(context=documents,Filter=ffilter)#fills few of the parameters here context

    # Invoke the LLM with the full message history
    new_message = parser.invoke(llm.invoke(full_prompt.invoke({"messages": messages})))
    # Extract the content of the latest message, which is the user's question


    # # Invoke the prompt with the correct arguments
    # new_message = parser.invoke(llm.invoke(prompt.invoke({"context": documents, "question": question})))

    return {"messages": [AIMessage(content=new_message)]}
# Define the graph
workflow = StateGraph(state_schema=RAGState)

# Add the 'model' node and connect it to the START of the graph
workflow.add_node("retriever",retrieveDocuments )
workflow.add_node("generate", generate_answer)
workflow.add_edge(START, "retriever")
workflow.add_edge("retriever", "generate")
workflow.add_edge("generate", END) # End the graph after the model responds


# Create/connect to the SQLite database
conn = sqlite3.connect("chatbot_memory.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)

app = workflow.compile(checkpointer=memory)
# --- Start a conversation with a unique ID ---
thread_id = "my_first_chat"
config2= {"configurable": {"thread_id": thread_id}}

  # First turn of the conversation
  # print("--- Turn 1 ---")
  # response1 = app.invoke({"messages": [HumanMessage(content="on the basis of these ahadith how should we pray janazah")]}, config=config)
  # print("Bot:", response1['messages'][-1].content)

  # # Second turn: The app will automatically load the previous messages
  # print("\n--- Turn 2 ---")
  # response2 = app.invoke({"messages": [HumanMessage(content="my nam is taha keep it engraved ")]}, config=config2)
  # print("Bot:", response2['messages'][-1].content)
  # Quran only
# response1 = app.invoke(
#     {
#         "messages": [HumanMessage(content="Give me the ayah about patience")],
#         "filters": "Quran"
#     },
#     config=config2
# )

# # Ahadith only
# response2 = app.invoke(
#     {
#         "messages": [HumanMessage(content="How to pray Janazah?")],
#         "filters": "Ahadith"
#     },
#     config=config2
# )

# Both
response3 = app.invoke(
    {
        "messages": [HumanMessage(content="also provide hadith")],
        "filters": "both"  # or "All"
    },
    config=config2
)

response3['messages'][-1].content.strip().removeprefix("```json").removesuffix("```").strip()



'{\n  "summary": "Allah commands justice, grace, and courtesy to close relatives, while forbidding indecency, wickedness, and aggression, instructing people to be mindful. Believers are urged to stand firm for Allah and bear true testimony, not letting hatred lead to injustice, but to be just as it is closer to righteousness. Furthermore, one should speak good words or remain silent, obedience is only in what is right, modesty is part of faith, actions are judged by intentions, and Islam is built on five pillars: testifying that there is no god but Allah and that Muhammad is the Messenger of Allah, performing the prayers, paying the Zakat, making the pilgrimage to the House, and fasting in Ramadan.",\n  "references": [\n    {\n      "type": "Quran",\n      "surah": "An-Nahl",\n      "ayahNumber": "90",\n      "ayahArabic": "۞ إِنَّ ٱللَّهَ يَأْمُرُ بِٱلْعَدْلِ وَٱلْإِحْسَٰنِ وَإِيتَآئِ ذِى ٱلْقُرْبَىٰ وَيَنْهَىٰ عَنِ ٱلْفَحْشَآءِ وَٱلْمُنكَرِ وَٱلْبَغْىِ ۚ يَعِظُكُمْ لَعَلَّكُمْ تَذَ

In [30]:
import json
cleaned = response3['messages'][-1].content.strip().removeprefix("```json").removesuffix("```").strip()
cleaned
data = json.loads(cleaned)
data# response2


{'summary': 'Allah commands justice, grace, and courtesy to close relatives, while forbidding indecency, wickedness, and aggression, instructing people to be mindful. Believers are urged to stand firm for Allah and bear true testimony, not letting hatred lead to injustice, but to be just as it is closer to righteousness. Furthermore, one should speak good words or remain silent, obedience is only in what is right, modesty is part of faith, actions are judged by intentions, and Islam is built on five pillars: testifying that there is no god but Allah and that Muhammad is the Messenger of Allah, performing the prayers, paying the Zakat, making the pilgrimage to the House, and fasting in Ramadan.',
 'references': [{'type': 'Quran',
   'surah': 'An-Nahl',
   'ayahNumber': '90',
   'ayahArabic': '۞ إِنَّ ٱللَّهَ يَأْمُرُ بِٱلْعَدْلِ وَٱلْإِحْسَٰنِ وَإِيتَآئِ ذِى ٱلْقُرْبَىٰ وَيَنْهَىٰ عَنِ ٱلْفَحْشَآءِ وَٱلْمُنكَرِ وَٱلْبَغْىِ ۚ يَعِظُكُمْ لَعَلَّكُمْ تَذَكَّرُونَ',
   'ayahEnglish': 'Ind

In [31]:
!pip install "uvicorn[standard]"


In [32]:
%pip install fastapi

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from fastapi import FastAPI
import uvicorn
import nest_asyncio
from typing import Optional
from fastapi.middleware.cors import CORSMiddleware




nest_asyncio.apply()


App = FastAPI()

App.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # or ["http://localhost:3000"]
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Allow Jupyter event loop to work with Uvicorn

@App.post("/chat")
def HadithBot(payload: dict):
    Question = payload.get("Question")
    filter_set = payload.get("filter")


    userID=payload.get("userID")
    BotResponse = app.invoke({"messages": [HumanMessage(content=Question)], "filters": filter_set}, config={"configurable": {"thread_id": userID}}
)
    cleaned = BotResponse['messages'][-1].content.strip().removeprefix("```json").removesuffix("```").strip()
    data=json.loads(cleaned)
    return data

uvicorn.run(App,host="127.0.0.1",port=8001)

INFO:     Started server process [23380]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23380]


In [ ]:
# from sentence_transformers import SentenceTransformer
#
# # Download from the 🤗 Hub
# matryoshka_dim = 786
# model = SentenceTransformer("omarelshehy/arabic-english-sts-matryoshka-v2.0", truncate_dim=matryoshka_dim)
# # Run inference
# sentences = [
#     "She enjoyed reading books by the window as the rain poured outside.",
#     "كانت تستمتع بقراءة الكتب بجانب النافذة بينما كانت الأمطار تتساقط في الخارج.",
#     "Reading by the window was her favorite thing, especially during rainy days."
# ]
# embeddings = model.encode(sentences)
#
# print(embeddings.shape)
# # [3, 1024]
#
# # Get the similarity scores for the embeddings
# similarities = model.similarity(embeddings, embeddings)
# print(similarities.shape)
# # [3, 3
# embeddings[1]


hadith chunkning of json file


In [4]:
!pip install jq

In [ ]:
from langchain_community.document_loaders.json_loader import JSONLoader
from langchain_core.
loader = JSONLoader(
    file_path="bukhari.json",
    
    jq_schema = ".hadiths[] | {id, bookId, chapterId, arabic, narrator: .english.narrator, text: .english.text}",
text_content=False #as we have a structureddata not a string 


)

bukhari = loader.load()
print(len(bukhari))   # should print number of hadiths
print(bukhari[1])
# from langchain_community.document_loaders.csv_loader import CSVLoader
# loader=CSVLoader(file_path="/content/all_hadiths_clean.csv",encoding="utf-8")
# docs=loader.load()
# print(len(docs))


SyntaxError: invalid syntax. Perhaps you forgot a comma? (1612753577.py, line 5)

BUkhari document formation and chunking

In [ ]:
import json
from langchain.docstore.document import Document

# Load JSON
with open("bukhari.json", "r", encoding="utf-8") as f:
    data = json.load(f)

bukhariDocs = []

for hadith in data["hadiths"]:
    chap = hadith['chapterId']  # numeric chapterId
    arabicBook = data['chapters'][chap-1]['arabic']  # check actual structure!
    engBook = data['chapters'][chap-1]['english']

    # Page content should be string, not dict
    page_content = f"Arabic: {hadith['arabic']}\nEnglish: {hadith['english']['text']}"

    # Metadata dictionary
    metadata = {
        'narrator': hadith['english']['narrator'],
        'chapEng': engBook,
        'chapArabic': arabicBook,
        'book': 'Sahih al-Bukhari'
    }

    # Create Document object
    doc = Document(page_content=page_content, metadata=metadata)
    bukhariDocs.append(doc)

print("Total documents created:", len(docs))
print(bukhariDocs[7276])  # preview first

# # If chapters exist, check first one
# if "hadiths" in data:
#     print(data[96])   # keys inside first chapter
#     print(json.dumps(data["hadiths"][7276], indent=2, ensure_ascii=False))
#     print(len(data["hadiths"]))  # pretty-print first chapter


Total documents created: 7277
page_content='Arabic: حَدَّثَنِي أَحْمَدُ بْنُ إِشْكَابٍ، حَدَّثَنَا مُحَمَّدُ بْنُ فُضَيْلٍ، عَنْ عُمَارَةَ بْنِ الْقَعْقَاعِ، عَنْ أَبِي زُرْعَةَ، عَنْ أَبِي هُرَيْرَةَ ـ رضى الله عنه ـ قَالَ قَالَ النَّبِيُّ صلى الله عليه وسلم ‏
"‏ كَلِمَتَانِ حَبِيبَتَانِ إِلَى الرَّحْمَنِ، خَفِيفَتَانِ عَلَى اللِّسَانِ، ثَقِيلَتَانِ فِي الْمِيزَانِ سُبْحَانَ اللَّهِ وَبِحَمْدِهِ، سُبْحَانَ اللَّهِ الْعَظِيمِ ‏"‏ ‏.‏
English: The Prophet (ﷺ) said, "(There are) two words which are dear to the Beneficent (Allah) and very light (easy) for the tongue (to say), but very heavy in weight in the balance. They are: ''Subhan Allah wa-bi hamdihi'' and ''Subhan Allah Al-`Azim." 

(see Hadith 6682).' metadata={'narrator': 'Narrated Abu Huraira:', 'chapEng': 'Oneness, Uniqueness of Allah (Tawheed)', 'chapArabic': 'كتاب التوحيد', 'book': 'Sahih al-Bukhari'}


In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

bukhariChunks=splitter.split_documents(bukhariDocs)
print(len(bukhariChunks))    

22379


    muslim document formation and chunking

In [4]:
import json
from langchain.docstore.document import Document

# Load JSON
with open("muslim.json", "r", encoding="utf-8") as f:
    data = json.load(f)

muslimDocs = []

for hadith in data["hadiths"]:
    chap = hadith['chapterId']  # numeric chapterId
    arabicBook = data['chapters'][chap-1]['arabic']  # check actual structure!
    engBook = data['chapters'][chap-1]['english']

    # Page content should be string, not dict
    page_content = f"Arabic: {hadith['arabic']}\nEnglish: {hadith['english']['text']}"

    # Metadata dictionary
    metadata = {
        'narrator': hadith['english']['narrator'],
        'chapEng': engBook,
        'chapArabic': arabicBook,
        'book': 'Sahih al-Muslim'
    }

    # Create Document object
    doc = Document(page_content=page_content, metadata=metadata)
    muslimDocs.append(doc)

print("Total documents created:", len(docs))
print(muslimDocs[7279])  # preview first

# # If chapters exist, check first one
# if "hadiths" in data:
#     print(data[96])   # keys inside first chapter
#     print(json.dumps(data["hadiths"][7276], indent=2, ensure_ascii=False))
#     print(len(data["hadiths"]))  # pretty-print first chapter


Total documents created: 7459
page_content='Arabic: حَدَّثَنِي حَرْمَلَةُ بْنُ يَحْيَى، أَخْبَرَنَا ابْنُ وَهْبٍ، أَخْبَرَنِي يُونُسُ، عَنِ ابْنِ شِهَابٍ، -
 وَهُوَ يَذْكُرُ الْحِجْرَ مَسَاكِنَ ثَمُودَ - قَالَ سَالِمُ بْنُ عَبْدِ اللَّهِ إِنَّ عَبْدَ اللَّهِ بْنَ عُمَرَ قَالَ مَرَرْنَا
 مَعَ رَسُولِ اللَّهِ صلى الله عليه وسلم عَلَى الْحِجْرِ فَقَالَ لَنَا رَسُولُ اللَّهِ صلى الله عليه وسلم
 ‏
"‏ لاَ تَدْخُلُوا مَسَاكِنَ الَّذِينَ ظَلَمُوا أَنْفُسَهُمْ إِلاَّ أَنْ تَكُونُوا بَاكِينَ حَذَرًا أَنْ يُصِيبَكُمْ مِثْلُ مَا
 أَصَابَهُمْ ‏"‏ ‏.‏ ثُمَّ زَجَرَ فَأَسْرَعَ حَتَّى خَلَّفَهَا ‏.‏
English: Salim b. 'Abdullah reported that 'Abdullah b. Umar said: We were passing along with Allah's Messenger (ﷺ) through the habitations of Hijr, and Allah's Messenger (ﷺ) said: Do not enter but weepingly the habitations of these persons who committed tyranny among themselves, lest the same calamity should fall upon you as it fell upon them. He then urged his mount to proceed quickly and pass through that

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

muslimChunks=splitter.split_documents(muslimDocs)
print(len(muslimChunks))    

19075


sunan nasai formation and chunking

In [ ]:
import json
from langchain.docstore.document import Document

# Load JSON
with open("tirmidhi.json", "r", encoding="utf-8") as f:
    data = json.load(f)

tirmidhiDocs = []

for hadith in data["hadiths"]:
    chap = hadith['chapterId'] 
     # numeric chapterId
    arabicBook = data['chapters'][chap-1]['arabic']  # check actual structure!
    engBook = data['chapters'][chap-1]['english']

    # Page content should be string, not dict
    page_content = f"Arabic: {hadith['arabic']}\nEnglish: {hadith['english']['text']}"

    # Metadata dictionary
    metadata = {
        'narrator': hadith['english']['narrator'],
        'chapEng': engBook,
        'chapArabic': arabicBook,
        'book': 'Jami` at-Tirmidhi',
    }

    # Create Document object
    doc = Document(page_content=page_content, metadata=metadata)
    tirmidhiDocs.append(doc)

print("Total documents created:", len(tirmidhiDocs))
print(tirmidhiDocs[4052])  # preview first

# # If chapters exist, check first one
# if "hadiths" in data:
#     print(data[96])   # keys inside first chapter
#     print(json.dumps(data["hadiths"][7276], indent=2, ensure_ascii=False))
#     print(len(data["hadiths"]))  # pretty-print first chapter


Total documents created: 4053
page_content='Arabic: حَدَّثَنَا هَارُونُ بْنُ مُوسَى بْنِ أَبِي عَلْقَمَةَ الْفَرْوِيُّ الْمَدَنِيُّ، قَالَ حَدَّثَنِي أَبِي، عَنْ هِشَامِ بْنِ سَعْدٍ، عَنْ سَعِيدِ بْنِ أَبِي سَعِيدٍ، عَنْ أَبِيهِ، عَنْ أَبِي هُرَيْرَةَ، رضى الله عنه أَنَّ رَسُولَ اللَّهِ صلى الله عليه وسلم قَالَ ‏
"‏ قَدْ أَذْهَبَ اللَّهُ عَنْكُمْ عُبِّيَّةَ الْجَاهِلِيَّةِ وَفَخْرَهَا بِالآبَاءِ مُؤْمِنٌ تَقِيٌّ وَفَاجِرٌ شَقِيٌّ وَالنَّاسُ بَنُو آدَمَ وَآدَمُ مِنْ تُرَابٍ ‏"‏ ‏.  هَذَا حَدِيثٌ حَسَنٌ صَحِيحٌ ‏.‏‏ وَهَذَا أَصَحُّ عِنْدَنَا مِنَ الْحَدِيثِ الأَوَّلِ ‏.‏ وَسَعِيدٌ الْمَقْبُرِيُّ قَدْ سَمِعَ مِنْ أَبِي هُرَيْرَةَ وَيَرْوِي عَنْ أَبِيهِ أَشْيَاءَ كَثِيرَةً عَنْ أَبِي هُرَيْرَةَ رضى الله عنه ‏.‏ وَقَدْ رَوَى سُفْيَانُ الثَّوْرِيُّ وَغَيْرُ وَاحِدٍ هَذَا الْحَدِيثَ عَنْ هِشَامِ ابْنِ سَعْدٍ عَنْ سَعِيدٍ الْمَقْبُرِيِّ عَنْ أَبِي هُرَيْرَةَ عَنِ النَّبِيِّ صلى الله عليه وسلم نَحْوَ حَدِيثِ أَبِي عَامِرٍ عَنْ هِشَامِ بْنِ سَعْدٍ ‏.‏
English: that the Messenger of Allah (ﷺ) sai

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

tirmidhiChunks=splitter.split_documents(tirmidhiDocs)
print(len(tirmidhiChunks))    

13663
